In [ ]:
import PyPDF2 as pdf
import sys, re, operator, itertools, shutil, time
import xlwings as xw
import numpy as np
import pandas as pd
import os, os.path
from os import listdir
import win32api
import pandas as pd

In [8]:
folder = "C:/Users/k_gag/OneDrive/Desktop/AC Report"
path = os.listdir(folder)
file = open("C:/Users/k_gag/OneDrive/Desktop/AC Report/ACReport2.pdf", 'rb')
 
ac = pdf.PdfReader(file)


text = ""
for page in ac.pages:
    text += page.extract_text()
    
all_substring = 'Air conditioning inspection report – Find an energy certificate – GOV .UK https://find-energy-certificate.service.gov .uk/energy-certificate'
cleaned_text = text.replace(all_substring, '')

print(cleaned_text)

29/05/2023, 18:24 Air conditioning inspection report – Find an energy certificate – GOV .UK
https://find-energy-certificate.service.gov .uk/energy-certificate/9666-5099-0025-0590-1895?print=true 1/30Air conditioning inspection report
Fitness First
Festival Leisure Park, Festival Way
BASILDON
SS14 3WBReport number
9666-5099-0025-0590-1895
This report expired on
19 November 2017This report has expired.
You can get a new certificate and report by visiting www.gov.uk/get-new-energy-certificate
Get help with reports for this property
If you need help getting a new report or if you know of other reports for this property that are not
listed here, contact the Department for Levelling Up, Housing and Communities (DLUHC).
dluhc.digital-services@levellingup.gov.uk
Telephone: 020 3829 0748!29/05/2023, 18:24 Air conditioning inspection report – Find an energy certificate – GOV .UK
https://find-energy-certificate.service.gov .uk/energy-certificate/9666-5099-0025-0590-1895?print=true 2/30Executive s

In [9]:
folder = "C:/Users/k_gag/OneDrive/Desktop/AC Report"
path = os.listdir(folder)
file = open("C:/Users/k_gag/OneDrive/Desktop/AC Report/ACReport2.pdf", 'rb')
 
ac = pdf.PdfReader(file)


text = ""
for page in ac.pages:
    text += page.extract_text()

import re
import pandas as pd


# Report Details
report_number = re.search(r'/energy-certificate/([^\n?]+)', cleaned_text)
report_number = report_number.group(1)

address = re.search(r'inspection of the air conditioning systems and \nassociated controls at ([^\n?]+)', cleaned_text)
if address:
    address = address.group(1).strip()

# Define the regular expression patterns for extracting the cooling plant details
patterns = {
    'Unit Identifier': r'Unit Identifier\s+([^\n]+)',
    'Location': r'Location of Cooling Plant\s+([^\n]+)',
    'Component Identifier': r'Component Identifier\s+([^\n]+)',
    'Rated Cooling Capacity': r'Rated Cooling Capacity \(kW\)\s+([\d.]+)',
    'Description': r'Description \(type/details\)\s+([^\n]+)',
    'Manufacturer': r'Manufacturer\s+([^\n]+)',
    'Model/Reference': r'Model/Reference\s+([^\n]+)',
    'Refrigerant Charge': r'Refrigerant Charge \(kg\)\s+([\d.]+)',
    'Refrigerant Type': r'Refrigerant Type\s+([^\n]+)',
    'Serial Number': r'Serial Number\s+([^\n]+)',
    'Year Plant Installed': r'Year Plant Installed\s+(\d+)',
}

# Initialize the AC inspection list to store details of multiple cooling plants
ac_insp = []

# Extract the cooling plant details from the text
for match in re.finditer(r'Cooling plant \d+', cleaned_text):
    cooling_plant_number = match.group().split()[-1]
    cooling_plant_details = {
        'Report Number': report_number,
        'Address': address,
        'Cooling Plant Number': cooling_plant_number,
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, cleaned_text)
        cooling_plant_details[key] = match.group(1) if match else ''

    ac_insp.append(cooling_plant_details)

    # Update cleaned_text to remove the previously matched details
    cleaned_text = cleaned_text[match.end():]

# Create a DataFrame from the ac_insp list
ac_insp_df = pd.DataFrame(ac_insp)

# Specify the desired column order
column_order = ['Report Number', 'Address', 'Cooling Plant Number', 'Unit Identifier', 'Location', 'Component Identifier', 'Rated Cooling Capacity', 'Description', 'Manufacturer', 'Model/Reference', 'Refrigerant Charge', 'Refrigerant Type', 'Serial Number', 'Year Plant Installed']

# Reorder the columns
ac_insp_df = ac_insp_df[column_order]

# Save the DataFrame as a CSV file
ac_insp_df.to_csv('C:/Users/k_gag/OneDrive/Desktop/AC Report/ac_inspection.csv', index=False)
